In [1]:
!pip install yfinance pandas openpyxl


In [37]:
import pandas as pd
import yfinance as yf
import numpy as np


In [38]:
# Fetch data
ticker = yf.Ticker("RELIANCE.NS")
income_raw = ticker.financials / 1_000_000  # Convert to millions


In [39]:
def format_financials(df_raw):
    df = df_raw.copy()
    df['Particular'] = df.index
    df.reset_index(drop=True, inplace=True)
    # Move 'Particular' to first column
    cols = ['Particular'] + [col for col in df.columns if col != 'Particular']
    df = df[cols]
    return df

income_statement = format_financials(income_raw)


In [40]:
income_statement

,Particular,2024-03-31 00:00:00,2023-03-31 00:00:00,2022-03-31 00:00:00,2021-03-31 00:00:00
0,Tax Effect Of Unusual Items,4065.48,-3185.73,8188.344,3659.07
1,Tax Rate For Calcs,0.0,0.0,0.0,0.0
2,Normalized EBITDA,1756750.0,1547650.0,1222880.0,893340.0
3,Total Unusual Items,16560.0,-14580.0,40860.0,106060.0
4,Total Unusual Items Excluding Goodwill,16560.0,-14580.0,40860.0,106060.0
5,Net Income From Continuing Operation Net Minor...,696210.0,662840.0,590440.0,491280.0
6,Reconciled Depreciation,508320.0,403030.0,297820.0,265720.0
7,Reconciled Cost Of Revenue,6745990.0,6718980.0,5316780.0,3390610.0
8,EBITDA,1773310.0,1533070.0,1263740.0,999400.0
9,EBIT,1264990.0,1130040.0,965920.0,733680.0


In [41]:
# Remove whitespace, lowercase for consistent matching
income_statement.columns = income_statement.columns.astype(str).str.strip()
income_statement['Particular'] = income_statement['Particular'].astype(str).str.strip().str.lower()


In [42]:
dates = list(income_statement.columns[1:])  # exclude 'Particular'


In [43]:
def safe_get(df, label):
    label = label.strip().lower()  # normalize
    row = df[df['Particular'] == label]
    if not row.empty:
        return pd.to_numeric(row.iloc[0, 1:].values, errors='coerce')  # force float
    else:
        return np.full(len(df.columns[1:]), np.nan)  # fill missing rows with NaN


In [44]:
revenue = safe_get(income_statement, 'total revenue')
other_income = safe_get(income_statement, 'other non operating income expenses')
total_income = revenue + other_income if not np.all(np.isnan(other_income)) else revenue
cogs = safe_get(income_statement, 'cost of revenue')
gross_profit = safe_get(income_statement, 'gross profit')
sgna = safe_get(income_statement, 'selling general and administration')
ebitda = safe_get(income_statement, 'ebitda')
da = safe_get(income_statement, 'reconciled depreciation')
ebit = safe_get(income_statement, 'ebit')
interest = safe_get(income_statement, 'interest expense')
ebt = safe_get(income_statement, 'pretax income')
tax = safe_get(income_statement, 'tax provision')
pat = safe_get(income_statement, 'net income')
basic_eps = safe_get(income_statement, 'basic eps')
diluted_eps = safe_get(income_statement, 'diluted eps')


In [45]:
final_income_statement = pd.DataFrame({
    'Particular': [
        'Revenue from operation', 'Other income', 'Total income',
        'COGS', 'Gross profit', 'SG&A', 'EBITDA', 'DA',
        'EBIT', 'INT', 'EBT', 'Tax', 'PAT', 'Basic EPS (Rs.)', 'Diluted EPS (Rs.)'
    ]
})

for i, date in enumerate(dates):
    final_income_statement[date] = [
        revenue[i],
        other_income[i],
        total_income[i],
        cogs[i],
        gross_profit[i],
        sgna[i],
        ebitda[i],
        da[i],
        ebit[i],
        interest[i],
        ebt[i],
        tax[i],
        pat[i],
        basic_eps[i],
        diluted_eps[i]
    ]


In [46]:
final_income_statement

,Particular,2024-03-31 00:00:00,2023-03-31 00:00:00,2022-03-31 00:00:00,2021-03-31 00:00:00
0,Revenue from operation,9.010640e+06,8.778350e+06,6.959630e+06,4.669240e+06
1,Other income,3.302000e+04,1.758000e+04,1.024000e+04,9.580000e+03
2,Total income,9.043660e+06,8.795930e+06,6.969870e+06,4.678820e+06
3,COGS,6.745990e+06,6.718980e+06,5.316780e+06,3.390610e+06
4,Gross profit,2.264650e+06,2.059370e+06,1.642850e+06,1.278630e+06
5,SG&A,3.714200e+05,3.167300e+05,2.482500e+05,1.967000e+05
6,EBITDA,1.773310e+06,1.533070e+06,1.263740e+06,9.994000e+05
7,DA,5.083200e+05,4.030300e+05,2.978200e+05,2.657200e+05
8,EBIT,1.264990e+06,1.130040e+06,9.659200e+05,7.336800e+05
9,INT,2.177200e+05,1.895800e+05,1.443800e+05,1.790700e+05
